In [35]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.structural import UnobservedComponents
import plotly.express as px
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf
from sklearn.linear_model import LinearRegression

pd.set_option('future.no_silent_downcasting', True)


# load data
df = pd.read_csv("data-clean.csv") 
column_names = ['gurkor', 'guitars', 'slingshots', 'stocks', 'sugar', 'water', 'tranquillity']

from scipy.signal import periodogram
from scipy.stats import linregress

def detect_seasonal_period(series, min_period=2, max_period=30, n_points=1000, detrend=True):
    series = series.dropna()
    if detrend:
        t = np.arange(len(series))
        slope, intercept, *_ = linregress(t, series.values)
        series = series - (slope * t + intercept)
    
    f, Pxx = periodogram(series, scaling='spectrum', nfft=n_points)
    periods = 1 / f
    valid = (periods >= min_period) & (periods <= max_period)
    if not np.any(valid):
        return None
    
    peak_idx = np.argmax(Pxx[valid])
    peak_period = periods[valid][peak_idx]
    return int(round(peak_period))

def extrapolate_trend(trend_series, gap_len, direction='forward'):
    trend_series = trend_series.dropna()
    n = len(trend_series)

    X = np.arange(n).reshape(-1, 1)
    y = trend_series.values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)

    if direction == 'forward':
        X_pred = np.arange(n, n + gap_len).reshape(-1, 1)
    elif direction == 'backward':
        X_pred = np.arange(-gap_len, 0).reshape(-1, 1)
    else:
        raise ValueError("direction must be 'forward' or 'backward'")

    y_pred = model.predict(X_pred).flatten()
    return np.array(pd.Series(y_pred, index=np.arange(gap_len)))


def extrapolated_increments(trend_series, gen_len, direction='forward'):

    trend_series = trend_series.dropna()
    n = len(trend_series)

    X = np.arange(n).reshape(-1, 1)
    y = trend_series.values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)

    slope = model.coef_[0, 0]

    if direction == 'forward':
        increments = slope * np.arange(1, gen_len + 1)
    elif direction == 'backward':
        increments = -slope * np.arange(gen_len, 0, -1)
    else:
        raise ValueError("direction must be 'forward' or 'backward'")
    
    return increments


def sample_seasonal(seasonal, period, gap_len, direction="forward"):
    if direction == "forward":
        gap_start = len(seasonal) % period
    elif direction == 'backward':
        gap_start = (-gap_len) % period
    else:
        raise ValueError("direction must be 'forward' or 'backward'")

    gap_end = gap_start + gap_len
    return np.array(seasonal[gap_start:gap_end])


def find_strong_turning_point(trend_series, direction='forward', window=5, min_slope_change=0.1, min_total_change=0.25):

    trend_series = trend_series.dropna()
    values = trend_series.values

    diff = pd.Series(np.diff(values)).rolling(window, center=True, min_periods=1).mean()

    second_diff = diff.diff().rolling(window, center=True, min_periods=1).mean()

    if direction == 'forward':
        indices = range(len(second_diff) - window, window, -1)
    else:
        indices = range(window, len(second_diff) - window)

    for i in indices:
        if np.sign(diff[i]) != np.sign(diff[i - 1]) and abs(second_diff[i]) > min_slope_change:
            if abs(values[i] - values[i - window]) > min_total_change:
                return i
    return None



In [36]:
def plot_interpolation(series, col, period=7, window=200, gap_index=0, plot_graphs=False, without_gap=None, ):

    # measure gap length
    is_na = series.isna()
    start_indices = series[is_na & ~is_na.shift(1).fillna(False).astype(bool)].index
    end_indices = series[is_na & ~is_na.shift(-1).fillna(False).astype(bool)].index

    # determine boundary indexes
    start, end = start_indices[gap_index], end_indices[gap_index]
    win_start = max(0, start - window)
    win_end = min(len(series), end + window + 1)
    sub_series = series[win_start:win_end]
    gap_len = sub_series.isna().sum()

    # decompose series before gaps
    stl_segment_before = sub_series.loc[win_start:start - 1].dropna()
    stl_before = STL(stl_segment_before, period=period, seasonal=7, robust=False)
    res_stl_before = stl_before.fit()

    turning_index_before = find_strong_turning_point(res_stl_before.trend, direction="forward")
    if turning_index_before is not None:
        trend_before = extrapolated_increments(res_stl_before.trend[turning_index_before:], sub_series.isna().sum(), direction="forward")

    seasonal_pattern_before = sample_seasonal(res_stl_before.seasonal, period=period, gap_len=sub_series.isna().sum(), direction="forward")

    residual_std_before = res_stl_before.resid.std()
    np.random.seed(40)
    noise_before = np.random.normal(0, residual_std_before, size=sub_series.isna().sum())


    # decompose series after gaps
    stl_segment_after = sub_series.loc[end+1:win_end+1].dropna()
    stl_after = STL(stl_segment_after, period=period, seasonal=7, robust=False)
    res_stl_after = stl_after.fit()

    turning_index_after = find_strong_turning_point(res_stl_after.trend, direction="backward")
    if turning_index_after is not None:
        trend_after = extrapolated_increments(res_stl_after.trend[:turning_index_after], sub_series.isna().sum(), direction="backward")

    seasonal_pattern_after = sample_seasonal(res_stl_after.seasonal, period=period, gap_len=sub_series.isna().sum(), direction="backward")
    
    residual_std_after = res_stl_after.resid.std()
    np.random.seed(42)
    noise_after = np.random.normal(0, residual_std_after, size=sub_series.isna().sum())   
    

    # kalman filter to get a growing base trend
    model = UnobservedComponents(sub_series, level='local linear trend', seasonal=period)
    res = model.fit(disp=False)

    filled = sub_series.copy()
    kalman_filled = res.smoothed_state[0, sub_series.isna()]
    filled[sub_series.isna()] = kalman_filled

    # dynamic customize weights
    inv_std_before = 1 / residual_std_before
    inv_std_after = 1 / residual_std_after
    total = inv_std_before + inv_std_after

    weights = {
        "before": 0.6,
        "after": 0.4
    }
    weights['before'] = inv_std_before / total
    weights['after'] = inv_std_after / total

    # add
    filled[sub_series.isna()] += (
        noise_before 
        * weights['before'] +
        noise_after
         * weights['after'] +
        seasonal_pattern_before 
        * weights['before'] +
        seasonal_pattern_after 
        * weights['after']
    )


    if turning_index_before is not None:
        filled[sub_series.isna()] += trend_before * weights['before'] / 2

    if turning_index_after is not None:
        filled[sub_series.isna()] += trend_after * weights['after'] / 2

    # plot line graphs
    if plot_graphs == True:
        gap_mask = sub_series.isna()
        x_gap = sub_series.index[gap_mask]
        y_gap = filled[gap_mask]

        np.random.seed(42)
        sigma = residual_std_before * weights['before'] + residual_std_after * weights['after']
        fluctuation = np.random.normal(0, 0.2 * sigma, size=len(y_gap))  
        local_sigma = sigma + fluctuation

        z = 1.96
        lower = y_gap - z * local_sigma
        upper = y_gap + z * local_sigma

        fig = px.line(x=sub_series.index, y=filled, title=f"{col} interpolation (with residual CI)")
        if without_gap is None:
            fig.add_scatter(x=sub_series.index, y=sub_series, mode='lines', name='original', line=dict(color='pink'))
        else:
            fig.add_scatter(x=sub_series.index, y=without_gap[win_start:win_end], mode='lines', name='original', line=dict(color='pink'))

        fig.add_scatter(x=x_gap, y=upper, mode='lines', line=dict(color='lightgray'), showlegend=False)
        fig.add_scatter(x=x_gap, y=lower, mode='lines', line=dict(color='lightgray'),
                      fill='tonexty', fillcolor='rgba(200,200,200,0.3)', showlegend=False)

        fig.show()


    series_copy = series.copy()
    series_copy.iloc[win_start:win_end] = filled
    return series_copy

for col in column_names:
    plot_interpolation(df[col], col, gap_index=0, plot_graphs=True)

In [37]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

WINDOW = 200
PERIOD = 7

periods = {
    'gurkor': 7, 
    'guitars': 7, 
    'slingshots': 7, 
    'stocks': 7, 
    'sugar': 7, 
    'water': 7, 
    'tranquillity': 7,
}

TEST_LEN = 2500
TEST_STEP = 20

warnings.filterwarnings("ignore", category=ConvergenceWarning)

def create_holdout(series, holdout_offset=400, holdout_length=50):
    
    is_missing = series.isna()
    gap_end = np.where(is_missing & ~is_missing.shift(-1, fill_value=False))[0][0]  # real gap end index

    holdout_start = gap_end + holdout_offset
    holdout_end = holdout_start + holdout_length

    # Avoid going out of bounds
    if holdout_end > len(series):
        raise ValueError("Holdout segment exceeds series length.")

    series_with_holdout = series.copy()
    holdout_true = series_with_holdout.iloc[holdout_start:holdout_end].copy()
    series_with_holdout.iloc[holdout_start:holdout_end] = np.nan

    return series_with_holdout, holdout_true, (holdout_start, holdout_end)

def validate_holdout(true_values, pred_values):
    mae = mean_absolute_error(true_values, pred_values)
    rmse = root_mean_squared_error(true_values, pred_values)
    return mae, rmse


for col in df.columns:
    period = detect_seasonal_period(df[col])
    print(f"{col}: detected period = {period}")


for col in column_names:
    print(f"\nProcessing column: {col}")
    
    original_series = df[col]

    mae_sum_linear, rmse_sum_linear = 0, 0
    mae_sum_custom, rmse_sum_custom = 0, 0

    start_offset = 250
    for offset in range(start_offset, start_offset + TEST_LEN, TEST_STEP):
        series_with_holdout, holdout_true, (start, end) = create_holdout(original_series, holdout_offset=offset, holdout_length=50)

        # Custom interpolation
        series_without_gap = series_with_holdout.copy()
        series_without_gap[start:end] = holdout_true
        filled_custom = plot_interpolation(series_with_holdout, col, period=periods[col], window=WINDOW, gap_index=1, plot_graphs=False, without_gap=series_without_gap)
        pred_custom = filled_custom.iloc[start:end]

        # Linear interpolation
        filled_linear = series_with_holdout.interpolate(method='linear')
        pred_linear = filled_linear.iloc[start:end]

        # Evaluate both
        mae_custom, rmse_custom = validate_holdout(holdout_true, pred_custom)
        mae_linear, rmse_linear = validate_holdout(holdout_true, pred_linear)

        mae_sum_custom += mae_custom
        rmse_sum_custom += rmse_custom
        mae_sum_linear += mae_linear
        rmse_sum_linear += rmse_linear

    print(f"Custom Interpolation - MAE: {mae_sum_custom:.4f}, RMSE: {rmse_sum_custom:.4f}")
    print(f"Linear Interpolation - MAE: {mae_sum_linear:.4f}, RMSE: {rmse_sum_linear:.4f}")
    print(f"Improvement - MAE: {(mae_sum_linear - mae_sum_custom ) / mae_sum_linear * 100:.4f}%, RMSE: {(rmse_sum_linear - rmse_sum_custom) / rmse_sum_linear * 100:.4f}%")



C:\Users\Xspec\AppData\Local\Temp\ipykernel_14272\3236117533.py:27: RuntimeWarning:

divide by zero encountered in divide



day: detected period = 29
gurkor: detected period = 29
guitars: detected period = 24
slingshots: detected period = 17
stocks: detected period = 21
sugar: detected period = 29
water: detected period = 16
tranquillity: detected period = 22

Processing column: gurkor
Custom Interpolation - MAE: 8.2288, RMSE: 9.8366
Linear Interpolation - MAE: 8.0534, RMSE: 9.6302
Improvement - MAE: -2.1782%, RMSE: -2.1425%

Processing column: guitars
Custom Interpolation - MAE: 23.3599, RMSE: 28.1973
Linear Interpolation - MAE: 22.8235, RMSE: 27.5465
Improvement - MAE: -2.3499%, RMSE: -2.3624%

Processing column: slingshots
Custom Interpolation - MAE: 11.7695, RMSE: 14.2263
Linear Interpolation - MAE: 11.5110, RMSE: 13.8824
Improvement - MAE: -2.2458%, RMSE: -2.4774%

Processing column: stocks
Custom Interpolation - MAE: 24.6502, RMSE: 29.6169
Linear Interpolation - MAE: 24.0741, RMSE: 28.9032
Improvement - MAE: -2.3928%, RMSE: -2.4693%

Processing column: sugar
Custom Interpolation - MAE: 9.6038, RMSE: 1